

# Título   Chequear ternaria

## Autor: Jose Chelquer
## Fecha de última modificación: 22/10/2025
## Descripción:

Chequear en un número de filas si fueron bien clasificadas en clase_ternaria

## Parámetros

< Descripción de cada uno de los parámetros que utiliza el job >


In [ ]:

usar_gdrive=False    # Poner en true si se va a ejecutar con archivos de google drive

casos_a_chequear=1000     # cantidad de filas a chequear

#verbose=0       # Sólo muestra cantidad de errores
verbose=1       # Sölo muestra los errores
#verbose=2       # Muestra todos los casos analizados

## Input

< Archivos de datos (csv.gz) con sus paths que van a consumirse por el job>

In [ ]:
dataset_path = '~/buckets/b1/datasets'
dataset_file = 'competencia_02.csv.gz'

## Output

< Archivos, bases de datos, modelos que va a generar el job>

In [ ]:
# No graba nada

## Procesos

### Paquetes necesarios

## Código del proceso

< Todo el código a partir de aquí debe poder ejecutarse sin necesidad de parametrizar nada>

In [ ]:
import os
import pandas as pd

In [ ]:
if usar_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
# leer el dataset

file_path = os.path.join(dataset_path, dataset_file)
print (f'Leyendo {file_path}')
# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.shape

Leyendo ~/buckets/b1/datasets/competencia_02.csv.gz


(4735593, 155)

In [ ]:
# Dado un mes en formato aaaamm y una cantidad de meses
# devuelve el mes a distancia meses de mes
def sig_mes(mes, meses):
  aaaa=mes//100
  mm=mes%100+meses
  aExtra=(mm-1)//12
  mm=mm-12*aExtra
  return (aaaa+aExtra)*100+mm


In [ ]:
columnas_de_interes = ['numero_de_cliente', 'foto_mes', 'clase_ternaria']

# Obtener una muestra aleatoria (puedes especificar el tamaño de la muestra con n)
df=df[columnas_de_interes]
muestra = df.sample(n=casos_a_chequear)
muestra['clase_ternaria'].value_counts()

clase_ternaria
CONTINUA    915
BAJA+2       43
BAJA+1       42
Name: count, dtype: int64

In [ ]:
def chequear_fila (fila, df):
  clase_ternaria=fila['clase_ternaria']
  numero_de_cliente=fila['numero_de_cliente']
  mes=fila['foto_mes']
  mes_mas_1=sig_mes(mes, 1)
  mes_mas_2=sig_mes(mes, 2)
  existe1=df[(df['numero_de_cliente']==numero_de_cliente) & (df['foto_mes']==mes_mas_1)].shape[0]>0
  existe2=df[(df['numero_de_cliente']==numero_de_cliente) & (df['foto_mes']==mes_mas_2)].shape[0]>0
  if clase_ternaria=='CONTINUA':
    ok=existe1 and existe2
  elif clase_ternaria=='BAJA+1':
    ok=not existe1
  elif clase_ternaria=='BAJA+2':
    ok=existe1 and not existe2
  else:
    ok=False
  mensaje=f'Clte: {numero_de_cliente} {clase_ternaria} ok: {ok}   ex1: {existe1} ex2: {existe2}'
  return (ok, mensaje)

In [ ]:
num=0
errores=0
for index, fila in muestra.iterrows():
  num=num+1
  ok, mensaje=chequear_fila(fila, df)
  if verbose==2 or (verbose==1 and not ok):
      print (num, mensaje)
  if not ok:
    errores=errores+1


In [ ]:
errores


0

In [ ]:
df.shape[0]

4735593

In [ ]:
conteo = df.groupby(['foto_mes', 'clase_ternaria']).size().reset_index(name='cantidad')

In [ ]:
print (conteo)

    foto_mes clase_ternaria  cantidad
0     201901         BAJA+1       688
1     201901         BAJA+2       718
2     201901       CONTINUA    123346
3     201902         BAJA+1       720
4     201902         BAJA+2       688
..       ...            ...       ...
88    202106         BAJA+2      1074
89    202106       CONTINUA    162894
90    202107         BAJA+1      1075
91    202107         BAJA+2    164077
92    202108         BAJA+1    165442

[93 rows x 3 columns]
